In [ ]:

# utility.ipynb を実行
%run utility.ipynb

读取文件

In [ ]:
# 读取CSV文件
# csv_file = 'BTCUSDT-5m-2023-04.csv'

stock_data = read_csv(symbol_name='BTCUSDT',  day='2023-04', interv='5m')

# 显示前几行数据
print(stock_data.head())

close_prices = stock_data['Close-5m']

close_prices

In [ ]:
close_price = close_prices[:150]
close_prices = close_price
close_prices

 创建要进行预测的数据集

In [ ]:

from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 初始化MinMaxScaler以规范化数据到0和1之间
scaler = MinMaxScaler(feature_range=(0, 1))

# 将`close_prices`数组转换为NumPy数组，并改变形状以适应scaler的要求。
# 这里`reshape(-1, 1)`表示将数据转换为一列，行数自动计算。
# 将 close_prices 转换为一个二维数组，每行一个数据点，因为 MinMaxScaler 需要二维输入。
close_prices = scaler.fit_transform(np.array(close_prices).reshape(-1, 1))

# 定义时间步长参数，指定使用过去100天的数据来预测下一天的价格
time_step = 100

# 调用之前定义的`create_dataset`函数来生成输入特征和标签数据集
# `X`将包含特征集，`y`将包含标签集
X, y = create_dataset(close_prices, time_step)

# 重新调整`X`的形状以适应后续模型的输入要求
# 这里`X.shape[0]`是样本数量，`X.shape[1]`是每个样本的时间步数量，最后的`1`表示每个时间步有一个特征
X = X.reshape(X.shape[0], X.shape[1], 1)

In [ ]:
X.shape
y.shape

根据你提供的 X.shape 结果 (1132, 100, 1)，我们可以解释这个三维数组的维度如下：

第一维（1132）：这表示生成的特征集 X 包含 1132 个样本。这些样本是从原始数据集中通过滑动窗口方法提取的，每个窗口包含 100 个时间步的数据。
第二维（100）：这表示每个样本由 100 个连续时间步组成。在你的案例中，这意味着每个样本包含 100 天的数据。这个数值与你在 create_dataset 函数中设置的 time_step 参数相匹配。
第三维（1）：这表示每个时间步只有一个特征。在股价预测的例子中，这个特征通常是股票的收盘价。这个维度是为了满足深度学习模型（如 LSTM）对数据形状的要求。

第一步：加载模型
在 PyTorch 中，你需要加载模型结构，然后加载状态字典（模型权重），这假设你在训练期间已经保存了模型的状态字典。

In [ ]:

import LSTMModel
import torch
# 假设模型的状态字典保存在 'model_state.pth'
model = LSTMModel()
model.load_state_dict(torch.load('model_state.pth'))
model.eval()

第二步：准备输入

In [ ]:
# 假设你的输入数据在 numpy 数组 'input_data' 中
input_tensor = torch.tensor(X, dtype=torch.float32)

第三步：进行预测

In [ ]:
# 做出预测
# 在 PyTorch 中，通常在进行推断时禁用梯度计算，这可以使推断运行更快。
with torch.no_grad():
    prediction = model(input_tensor)

# 反转预测
train_predict = scaler.inverse_transform(prediction)
y = scaler.inverse_transform(y.reshape(y.shape[0],1))

对每个样本进行预测下一天的股价

In [ ]:
train_predict.shape

画图

In [ ]:
import matplotlib.pyplot as plt

predicted_data = train_predict
actual_data = y 


# 确保预测数据和实际数据长度一致
dates = range(len(actual_data))  # 如果有具体日期可以用具体日期

plt.figure(figsize=(10, 5))
plt.plot(dates, actual_data, label='Actual Data', color='blue', marker='x')
plt.plot(dates, predicted_data, label='Predicted Data', color='red', marker='.')


plt.title('Comparison of Actual and Predicted Values')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

actual_data[:50]
predicted_data[:50]